In [43]:
# Encoder Class: Prepares the input image data.
# Attention Class: Computes attention scores between decoder hidden states and encoder outputs.
# OneStepDecoder Class: Handles decoding one step at a time, integrating attention.
# Decoder Class: Utilizes OneStepDecoder for decoding entire sequences.
# encoder_decoder Class: Combines Encoder and Decoder into a single model.

In [51]:
# Compile Model:

# Define optimizer, loss function, and metrics for training.
# Compile the encoder_decoder model.
# Fit Model (model_1.fit):

# Train the model using training data (train_inp, dec_inp, dec_op).
# Validate using testing data (test_inp, dec_inp_cv, dec_op_cv).
# Callbacks such as ModelCheckpoint and ReduceLROnPlateau are used for monitoring and optimization.
# Step 4: Generating Captions
# Generate Caption Example:
# After training, to generate a caption for a new image:
# Initialize with a start token (<start>).
# Pass through the decoder (model_1.decoder) iteratively.
# Use the OneStepDecoder to predict each subsequent token until an end token (<end>) or maximum length is reached.
# Flow of Classes in Execution
# Data Preparation Phase:

# Tokenizer is used to prepare text data (findings_total) for training.
# Sequences are padded to ensure consistent input size.
# Model Definition Phase:

# Encoder (Encoder): Processes image features (train_inp) to create initial context.
# Attention (Attention): Calculates attention weights based on decoder state and encoder output.
# One Step Decoder (OneStepDecoder): Generates predictions and updates states at each timestep.
# Decoder (Decoder): Integrates OneStepDecoder to generate entire output sequence.
# Training Phase:

# Model (encoder_decoder): Integrates encoder and decoder logic for end-to-end training.
# Callbacks: Monitors and optimizes training progress (ModelCheckpoint, ReduceLROnPlateau).
# Caption Generation Phase:

# Inference: Uses trained model to predict captions for new images.
# Tokenization: Converts predicted token IDs back to words using reverse mapping (imp1).
# This structured flow ensures each component (Encoder, Attention, Decoder) operates cohesively to generate accurate captions based on image features, demonstrating a clear progression from data preparation through model execution and application.








In [1]:


import os
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from PIL import Image
import tensorflow as tf
from keras.regularizers import l2
import cv2 as cv
from keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.backend import expand_dims
from tensorflow import concat
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import TimeDistributed
import cv2

from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM,Layer,Dropout,GRU
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import repeat
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')


print("done importing")

2024-06-26 23:35:10.818209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 23:35:10.818300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 23:35:10.843560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-26 23:35:10.901725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 23:35:11.843125: W tensorflow/compiler/tf2

done importing


In [3]:
os.listdir()

['encod.zip',
 'Test_Data.csv',
 'CV_Data.csv',
 'Image_features_attention.pickle',
 'model_with_accu.ipynb',
 '.ipynb_checkpoints',
 'LLM_main1.ipynb',
 'log.zip',
 'attention_v8_(1) (1).ipynb',
 'log',
 'Untitled.ipynb',
 'checkpoint',
 'NLMCXR_png',
 'untitled3.py',
 'processed.pkl',
 'Train_Data.csv',
 'LLM1.ipynb',
 'encoder_decoder_model.keras',
 'NLMCXR_reports',
 't1.pickle',
 'encod',
 'chexnet1.pptx']

In [2]:
data = pd.read_pickle('/home/professor/Downloads/fromgit/single_image_final_1024.pkl')


In [3]:
X_train, X_test = train_test_split(data.values , test_size = 0.2 )

In [4]:
X_train.shape

(3035, 8)

In [5]:
data.head(5)

,filename,uid,findings,impression,image_features,findings_total,dec_ip,dec_op
0,1_IM-0001-4001.dcm.png,1,the cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,"[[0.00026644140598364174, 0.001953634200617671...",<start> the cardiac silhouette and mediastinum...,<start> the cardiac silhouette and mediastinum...,the cardiac silhouette and mediastinum size ar...
1,2_IM-0652-1001.dcm.png,2,borderline cardiomegaly. midline sternotomy . ...,No acute pulmonary findings.,"[[0.00022730983619112521, 0.001471916912123561...",<start> borderline cardiomegaly. midline stern...,<start> borderline cardiomegaly. midline stern...,borderline cardiomegaly. midline sternotomy . ...
2,3_IM-1384-1001.dcm.png,3,"no displaced rib fractures, pneumothora, or pl...","No displaced rib fractures, pneumothorax, or p...","[[0.00037261098623275757, 0.001325369114056229...","<start> no displaced rib fractures, pneumothor...","<start> no displaced rib fractures, pneumothor...","no displaced rib fractures, pneumothora, or pl..."
3,4_IM-2050-1001.dcm.png,4,there are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...,"[[7.35483699827455e-05, 0.0012481726007536054,...",<start> there are diffuse bilateral interstiti...,<start> there are diffuse bilateral interstiti...,there are diffuse bilateral interstitial and a...
4,5_IM-2117-1003002.dcm.png,5,the cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.,"[[0.0005863018450327218, 0.0017833115998655558...",<start> the cardiomediastinal silhouette and p...,<start> the cardiomediastinal silhouette and p...,the cardiomediastinal silhouette and pulmonary...


In [47]:
print(data.iloc[0]["image_features"].shape)


(1, 2048)


In [8]:
maindir = '/home/professor/Downloads/fromgit/NLMCXR_png/'


In [9]:
data['filename'] = data['image1'].str.replace('/content/png/', maindir)



In [10]:
data.iloc[0]['image1']

'/home/professor/Downloads/fromgit/NLMCXR_png/CXR3556_IM-1741-1001-0001.png'

In [6]:
X_train = X_train[:-19, :]
X_test = X_test[:-18, :]

In [12]:
X_train.shape

(2650, 8)

In [13]:
X_test.shape

(650, 8)

In [14]:
data.columns


Index(['patient_id', 'image1', 'image2', 'findings', 'image_features',
       'findings_total', 'dec_ip', 'dec_op'],
      dtype='object')

In [15]:

t1 = Tokenizer( filters='!"#$%&()*+,-/:;=?@[\\]^_`{|}~\t\n',oov_token='OOV')
t1.fit_on_texts(X_train[:,5])
vocab_size_imp = len(t1.word_index) + 1

dec_inp = t1.texts_to_sequences(X_train[:,6])

dec_inp = pad_sequences(dec_inp, maxlen=76, padding='post')

dec_inp_cv = t1.texts_to_sequences(X_test[:,6])

dec_inp_cv = pad_sequences(dec_inp_cv, maxlen=76, padding='post')

dec_op = t1.texts_to_sequences(X_train[:,7])

dec_op = pad_sequences(dec_op, maxlen=76, padding='post')

dec_op_cv = t1.texts_to_sequences(X_test[:,7])

dec_op_cv = pad_sequences(dec_op_cv, maxlen=76, padding='post')

In [16]:

# example
sequences = [
    [1, 2, 3],
    [4, 5],
    [6, 7, 8, 9]
]

padded_sequences = pad_sequences(sequences, maxlen=6, padding='post')
padded_sequences

array([[1, 2, 3, 0, 0, 0],
       [4, 5, 0, 0, 0, 0],
       [6, 7, 8, 9, 0, 0]], dtype=int32)

In [52]:
t1.word_index

{'OOV': 1,
 'the': 2,
 'no': 3,
 'are': 4,
 'is': 5,
 '<start>': 6,
 '<end>': 7,
 'and': 8,
 'pleural': 9,
 'or': 10,
 'of': 11,
 'normal': 12,
 'there': 13,
 'heart': 14,
 'lungs': 15,
 'focal': 16,
 'size': 17,
 'within': 18,
 'in': 19,
 'pneumothora': 20,
 'pulmonary': 21,
 'effusion': 22,
 'normal.': 23,
 'pneumothora.': 24,
 'effusion.': 25,
 'limits.': 26,
 'right': 27,
 'silhouette': 28,
 'mediastinal': 29,
 'airspace': 30,
 'clear.': 31,
 'cardiomediastinal': 32,
 'left': 33,
 'acute': 34,
 'clear': 35,
 'consolidation': 36,
 'lung': 37,
 'with': 38,
 'structures': 39,
 'mediastinum': 40,
 'unremarkable.': 41,
 'changes': 42,
 'thoracic': 43,
 'bony': 44,
 'a': 45,
 'stable': 46,
 'degenerative': 47,
 'spine.': 48,
 'without': 49,
 'mild': 50,
 'large': 51,
 'appear': 52,
 'osseous': 53,
 'contours': 54,
 'cardiac': 55,
 'limits': 56,
 'consolidation.': 57,
 'size.': 58,
 'calcified': 59,
 '.': 60,
 'evidence': 61,
 'vascularity': 62,
 'for': 63,
 'disease.': 64,
 'visualized':

In [18]:
print(dec_inp[0])

[  6 162  15   4  35  82  55   8  29 217   4  23  21  74   5  23   3  20
  10   9  25   3  34  44  67 419  13   4   3 198 101 231  10 837 966 101
 247 105 184  52  23   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


In [19]:

with open('t1.pickle', 'wb') as handle:
    pickle.dump(t1,handle)

In [48]:
vocab_size_imp


1830

In [21]:
imp1 = {}
imp2 = {}
for key,value in t1.word_index.items():
  imp1[value] = key
  imp2[key] = value

In [49]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,units):
        super().__init__()
        self.units = units
        self.dense = Dense(self.units,name = 'Enc_dense')

    
    def call(self,img):
      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      #enc_out = self.maxpool(tf.expand_dims(img,axis = 2))
      enc_out = self.dense(img)
      print("Enc_out = ", enc_out)
      return enc_out


    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      self.batch_size  = batch_size

      self.enc_h =tf.zeros((self.batch_size, self.units))

      #self.enc_c = tf.zeros((self.batch_size, self.lstm_size))
      return self.enc_h


In [23]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,att_units):
    # Please go through the reference notebook and research paper to complete the scoring functions
    super().__init__()

    self.att_units = att_units

    self.w1 =  tf.keras.layers.Dense( self.att_units , name = 'w1')
    self.w2 =  tf.keras.layers.Dense( self.att_units,name = 'w2')
    self.v =  tf.keras.layers.Dense(1,name = 'v')
    
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    self.decoder_hidden_state = decoder_hidden_state
    self.encoder_output = encoder_output


    self.decoder_hidden_state = tf.expand_dims(self.decoder_hidden_state,axis = 1)
    score = self.v(tf.nn.tanh(
              self.w1(self.decoder_hidden_state) + self.w2(self.encoder_output)))
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * self.encoder_output
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector,attention_weights





In [24]:
class OneStepDecoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units  ,att_units):
      
      # Initialize decoder embedding layer, LSTM and any other objects needed
      super().__init__()
      self.tar_vocab_size = tar_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.att_units = att_units
      self.dec_emb = Embedding(tar_vocab_size,embedding_dim,trainable = True , name = 'dec_embb')
      self.dec_lstm = GRU(self.dec_units, return_state=True, return_sequences=True, name="Decoder_LSTM")
      self.dense   = Dense(self.tar_vocab_size, name = 'one_dec')
      self.attention=Attention( self.att_units)
      self.d1 = Dropout(0.3,name = 'd1')
      self.d2 = Dropout(0.3,name = 'd2')
      self.d3 = Dropout(0.3,name = 'd3')

  @tf.function
  def call(self,input_to_decoder, encoder_output, state_h):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    self.input_to_decoder = input_to_decoder
    self.encoder_output = encoder_output
    self.state_h = state_h

    #A
    target_embedd           = self.dec_emb (self.input_to_decoder)     #(batch_size,1,embedingdim)
    #B
    target_embedd = self.d1(target_embedd)

    context_vector,attention_weights=self.attention(self.state_h,self.encoder_output) #context vector shape = (batch_size,att_units)
    #C
    concated = tf.concat([  tf.expand_dims(context_vector, 1),target_embedd], -1)
    concated = self.d2(concated)

    #D
    lstm_output, hs      = self.dec_lstm(concated, initial_state=self.state_h)

    lstm_output = tf.reshape(lstm_output, (-1, lstm_output.shape[2]))
    lstm_output = self.d3(lstm_output)
    #E
    op = self.dense(lstm_output)
    #op = tf.squeeze(op,[1])
    return op,hs,attention_weights,context_vector




In [25]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super().__init__()
      self.out_vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.att_units = att_units
      
      self.onestep = OneStepDecoder(self.out_vocab_size,self.embedding_dim ,self.input_length,self.dec_units,self.att_units)

    @tf.function
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state):


        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook

        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array

        all_outputs = tf.TensorArray(tf.float32,size =input_to_decoder.shape[1],name = 'output_arrays' )
        self.input_to_decoder = input_to_decoder
        self.encoder_output = encoder_output
        self.decoder_hidden_state = decoder_hidden_state

        for timestep in tf.range(input_to_decoder.shape[1]):
          op,hs,attention_weights,context_vector = self.onestep(self.input_to_decoder[:,timestep:timestep+1], self.encoder_output, self.decoder_hidden_state)
          self.decoder_hidden_state = hs
          all_outputs = all_outputs.write(timestep,op)
        all_outputs = tf.transpose(all_outputs.stack(),[1,0,2])
        return all_outputs


In [26]:
class encoder_decoder(tf.keras.Model):
  #def __init__(self,#params):
    #Intialize objects from encoder decoder
  def __init__(self,out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units,batch_size,units):

        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        super().__init__()

        self.units = units
        self.out_vocab_size = out_vocab_size
        self.embedding_size_d = embedding_size_d
        self.lstm_size_d = lstm_size_d
        self.input_length_d = input_length_d
        self.batch_size = batch_size
        self.att_units = att_units

        self.encoder = Encoder(self.units)
        print("OUT_VOCAB_SIZE = ",out_vocab_size, embedding_size_d,input_length_d,lstm_size_d,att_units)

        self.decoder = Decoder(out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units )
        #self.dense   = TimeDistributed(Dense(self.out_vocab_size, activation='softmax'))
        self.dense   = Dense(self.out_vocab_size,name = 'enc_dec_dense')



  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    self.inputs,self.outputs = data[0], data[1]
    print("="*20, "ENCODER", "="*20)
    self.encoder_h= self.encoder.initialize_states(self.batch_size)
    self.encoder_output = self.encoder(self.inputs)
    print("-"*27)
    print("ENCODER ==> OUTPUT SHAPE",self.encoder_output.shape)
    print("ENCODER ==> HIDDEN STATE SHAPE",self.encoder_h.shape)
    print("="*20, "DECODER", "="*20)
    output= self.decoder(self.outputs,self.encoder_output,self.encoder_h)
    print("-"*27)
    print("FINAL OUTPUT SHAPE",output.shape)
    print("="*50)
    print("output = ", self.dense(output))

    return output






In [27]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  

  return tf.reduce_mean(loss_)
  #out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units,batch_size)

2024-06-25 22:14:34.312844: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-06-25 22:14:34.312899: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: professor-Aspire-A715-42G
2024-06-25 22:14:34.312911: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: professor-Aspire-A715-42G
2024-06-25 22:14:34.313098: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.183.1
2024-06-25 22:14:34.313130: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.171.4
2024-06-25 22:14:34.313136: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:244] kernel version 535.171.4 does not match DSO version 535.183.1 -- cannot find working devices in this 

In [28]:
!mkdir checkpoint


mkdir: cannot create directory ‘checkpoint’: File exists


In [29]:
cwd = os.getcwd()

checkpoint_filepath = cwd + '/' + 'checkpoint' + '/'


In [30]:
checkpoint_filepath


'/home/professor/Downloads/fromgit/checkpoint/'

In [31]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    verbose = 1,
    mode='min',
    save_best_only=True)

In [32]:
tf.keras.backend.clear_session()


In [33]:

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8,mode = 'min',verbose = 1,
                              patience=2, min_lr=0.0001)

In [34]:
model_1 = encoder_decoder(vocab_size_imp,300,76,256,64,50,256)
optimizer = tf.keras.optimizers.Adam()
model_1.compile(optimizer=optimizer,loss=loss_function, metrics = "accuracy")


OUT_VOCAB_SIZE =  1830 300 76 256 64


In [41]:
encoder = Encoder(units=256)
batch_size = 50

encoder_hidden_state = Encoder.initialize_states(encoder,batch_size)
print(encoder_hidden_state.shape)  # Output: (50, 256)


(50, 256)


In [42]:
attention = Attention(att_units=64)
decoder_hidden_state = tf.random.normal((50, 256))
encoder_output = tf.random.normal((50, 10, 256))
context_vector, attention_weights = attention(decoder_hidden_state, encoder_output)
print(context_vector.shape)  # Output: (50, 256)
print(attention_weights.shape)  # Output: (50, 10, 1)


(50, 256)
(50, 10, 1)


In [35]:
train_inp = np.vstack(X_train[:,4]).astype(np.float64)  # Use np.float64 for double-precision floats
test_inp = np.vstack(X_test[:,4]).astype(np.float32)  # Use np.float32 for single-precision floats (if memory efficiency is a concern)


In [50]:
model_1.fit([train_inp,dec_inp ],dec_op ,validation_data= ([test_inp, dec_inp_cv],dec_op_cv),batch_size= 50,epochs  = 1,callbacks=[reduce_lr,model_checkpoint_callback] , shuffle=True)

53/53 [==============================] - ETA: 0s - loss: 2.1220 - accuracy: 0.0234
Epoch 1: val_loss improved from 2.20909 to 2.18705, saving model to /home/professor/Downloads/fromgit/checkpoint/
==================== ENCODER ====================
---------------------------
ENCODER ==> OUTPUT SHAPE (None, 256)
ENCODER ==> HIDDEN STATE SHAPE (50, 256)
==================== DECODER ====================
---------------------------
FINAL OUTPUT SHAPE (50, 76, 1830)
output =  Tensor("encoder_decoder/enc_dec_dense/BiasAdd:0", shape=(50, 76, 1830), dtype=float32)
==================== ENCODER ====================
---------------------------
ENCODER ==> OUTPUT SHAPE (None, 256)
ENCODER ==> HIDDEN STATE SHAPE (50, 256)
==================== DECODER ====================
---------------------------
FINAL OUTPUT SHAPE (None, 76, 1830)
output =  Tensor("enc_dec_dense/StatefulPartitionedCall:0", shape=(None, 76, 1830), dtype=float32)
==================== ENCODER ====================
-------------------

INFO:tensorflow:Assets written to: /home/professor/Downloads/fromgit/checkpoint/assets


53/53 [==============================] - 44s 837ms/step - loss: 2.1220 - accuracy: 0.0234 - val_loss: 2.1870 - val_accuracy: 0.0257 - lr: 0.0010


In [39]:

model_1.load_weights(checkpoint_filepath)


2024-06-25 21:19:27.927445: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open /home/professor/Downloads/fromgit/checkpoint/: FAILED_PRECONDITION: /home/professor/Downloads/fromgit/checkpoint; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [40]:
#https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
#https://github.com/vysakh10/Medical-Report-Generation-using-Deep-Learning/blob/master/Attention_Onestep_Mod.ipynb
def beam(sentence):
  """ Thi function predicts the sentence using beam search """

  initial_state=model_1.layers[0].initialize_states(1)
  encoder_output= model_1.layers[0](sentence)
  result = ''

  sequences = [['<start>' ,initial_state, 0]]

  decoder_hidden_state = initial_state

  #cur_vec = np.ones((1, 1)) * imp2['<start>']
  #cur_vec = np.array([[1]])
  finished_seq = []
  beam_width = 3

  for i in range(76):
    all_candidates = []
    new_seq = []
    for s in sequences:

      cur_vec = np.reshape(imp2[s[0].split(" ")[-1]],(1,1))
      decoder_hidden_state = s[1]
      op,hs,attention_weights,context_vector = model_1.layers[1].onestep(cur_vec, encoder_output, decoder_hidden_state)
      op = tf.nn.softmax(op)
      top3 = np.argsort(op).flatten()[-beam_width:]
      for t in top3:

         candidates = [s[0] + ' '+ imp1[t], hs,s[2]-np.log(np.array(op).flatten()[t])]
         all_candidates.append(candidates)
    sequences = sorted(all_candidates, key = lambda l: l[2])[:beam_width]

    count = 0
    for s1 in sequences:
      if s1[0].split(" ")[-1] == '<end>':


        s1[2] = s1[2]/len(s1[0])   # normalized
        finished_seq.append([s1[0], s1[1],s1[2]])
        count+=1
      else:

        new_seq.append([s1[0], s1[1],s1[2]])

    beam_width -= count
    sequences = new_seq
    if not sequences:
      break
    else:
      continue

  if len(finished_seq) >0:
    sequences = finished_seq[-1]
    return sequences[0]
  else:
    return new_seq[-1][0]


In [41]:
num = 16 #random test sample
im_o = np.vstack(X_test[:,4][num]).astype(float)
tex_o = X_test[:,6][num]

print("Original Sentence is : " , tex_o)
print("Predicted Sentence is : ",beam(im_o))
# img = cv2.imread(X_test[:,1][num], cv2.IMREAD_UNCHANGED)
# cv2_imshow(img)
# img = cv2.imread(X_test[:,2][num], cv2.IMREAD_UNCHANGED)
# cv2_imshow(img)


Original Sentence is :  <start> the heart is normal in size and contour. there is no mediastinal widening. the lungs are clear bilaterally. no large pleural effusion or pneumothora. fractures of the posterior left th, th, and th ribs, age-indeterminate.
Predicted Sentence is :  <start> the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the is


In [39]:
import nltk.translate.bleu_score as bleu

#https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
b1 = 0
b2 = 0
b3 = 0
b4 = 0


for i in range(X_test.shape[0]):
  im_o = np.vstack(X_test[:,4][i]).astype(float)
  pred = beam(im_o)
  org= X_test[:,5][i]




  b1 =  b1 + bleu.sentence_bleu([org.split()], pred.split() ,weights=(1, 0, 0, 0))
  b2 =  b2 + bleu.sentence_bleu([org.split()], pred.split(),weights=(0.5, 0.5, 0, 0))
  b3 =  b3 + bleu.sentence_bleu([org.split()], pred.split() ,weights=(0.33, 0.33, 0.33, 0))
  b4 =  b4 + bleu.sentence_bleu([org.split()], pred.split() ,weights=(0.25, 0.25, 0.25, 0.25))



print("Bleu1 score is : ",b1/X_test.shape[0])
print("Bleu2 score is : ",b2/X_test.shape[0])
print("Bleu3 score is : ",b3/X_test.shape[0])
print("Bleu4 score is : ",b4/X_test.shape[0])


Bleu1 score is :  0.32272852431776494
Bleu2 score is :  0.16857199334071807
Bleu3 score is :  0.0933629868126091
Bleu4 score is :  0.043901071057910905


In [40]:
c = set()
for num in range(650):
    im_o = np.vstack(X_train[:, 4][num]).astype(float)
    tex_o = X_test[:, 6][num]
    if(num%100 == 0):
      print("Sample Number:", num)
    # print("Sample Number:", num)
    # print("Original Sentence is:", tex_o)
    # print("Predicted Sentence is:", beam(im_o))
    # print("The X-Rays are:")

    # img1 = cv2.imread(X_test[:, 1][num], cv2.IMREAD_UNCHANGED)
    # img2 = cv2.imread(X_test[:, 2][num], cv2.IMREAD_UNCHANGED)

    c.add(beam(im_o))
    # print(count)
    # cv2_imshow(img1)
    # cv2_imshow(img2)
    # print("\n" + "-"*50 + "\n")

Sample Number: 0
Sample Number: 100
Sample Number: 200
Sample Number: 300
Sample Number: 400
Sample Number: 500
Sample Number: 600


In [41]:


import os
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from PIL import Image
import tensorflow as tf
from keras.regularizers import l2
import cv2 as cv
from keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.backend import expand_dims
from tensorflow import concat
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import TimeDistributed
import cv2

from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM,Layer,Dropout,GRU
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import repeat
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')


print("done importing")

data = pd.read_pickle('/home/professor/Downloads/fromgit/processed.pkl')
X_train, X_test = train_test_split(data.values , test_size = 0.2 )
data.head(5)
patient_id	image1	image2	findings	image_features	findings_total	dec_ip	dec_op
0	CXR3556	/content/png/CXR3556_IM-1741-1001-0001.png	/content/png/CXR3556_IM-1741-1001-0002.png	the lungs are clear. there is no pleural effus...	[[0.00041582860285416245, 0.001570420921780169...	<start> the lungs are clear. there is no pleur...	<start> the lungs are clear. there is no pleur...	the lungs are clear. there is no pleural effus...
2	CXR32	/content/png/CXR32_IM-1511-1001.png	/content/png/CXR32_IM-1511-4001.png	the heart is normal in size. the mediastinum i...	[[0.0005354544264264405, 0.0019668142776936293...	<start> the heart is normal in size. the media...	<start> the heart is normal in size. the media...	the heart is normal in size. the mediastinum i...
4	CXR260	/content/png/CXR260_IM-1090-1001.png	/content/png/CXR260_IM-1090-2001.png	lungs are clear bilaterally. cardiac and media...	[[0.0002745636156760156, 0.0018877924885600805...	<start> lungs are clear bilaterally. cardiac a...	<start> lungs are clear bilaterally. cardiac a...	lungs are clear bilaterally. cardiac and media...
5	CXR1301	/content/png/CXR1301_IM-0198-1001.png	/content/png/CXR1301_IM-0198-2001.png	heart size within normal limits, stable medias...	[[0.0005874697235412896, 0.0018448150949552655...	<start> heart size within normal limits, stabl...	<start> heart size within normal limits, stabl...	heart size within normal limits, stable medias...
6	CXR1921	/content/png/CXR1921_IM-0598-1001.png	/content/png/CXR1921_IM-0598-2001.png	redemonstration of moderately-inflated lungs, ...	[[0.00029747304506599903, 0.0014215346891433, ...	<start> redemonstration of moderately-inflated...	<start> redemonstration of moderately-inflated...	redemonstration of moderately-inflated lungs, ...


t1 = Tokenizer( filters='!"#$%&()*+,-/:;=?@[\\]^_`{|}~\t\n',oov_token='OOV')
t1.fit_on_texts(X_train[:,5])
vocab_size_imp = len(t1.word_index) + 1

dec_inp = t1.texts_to_sequences(X_train[:,6])

dec_inp = pad_sequences(dec_inp, maxlen=76, padding='post')

dec_inp_cv = t1.texts_to_sequences(X_test[:,6])

dec_inp_cv = pad_sequences(dec_inp_cv, maxlen=76, padding='post')

dec_op = t1.texts_to_sequences(X_train[:,7])

dec_op = pad_sequences(dec_op, maxlen=76, padding='post')

dec_op_cv = t1.texts_to_sequences(X_test[:,7])

dec_op_cv = pad_sequences(dec_op_cv, maxlen=76, padding='post')


with open('t1.pickle', 'wb') as handle:
    pickle.dump(t1,handle)
imp1 = {}
imp2 = {}
for key,value in t1.word_index.items():
  imp1[value] = key
  imp2[key] = value

class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,units):
        super().__init__()
        self.units = units
        self.dense = Dense(self.units,name = 'Enc_dense')

    
    def call(self,img):
      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      #enc_out = self.maxpool(tf.expand_dims(img,axis = 2))
      enc_out = self.dense(img)
      return enc_out


    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      self.batch_size  = batch_size

      self.enc_h =tf.zeros((self.batch_size, self.units))

      #self.enc_c = tf.zeros((self.batch_size, self.lstm_size))
      return self.enc_h
         
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,att_units):
    # Please go through the reference notebook and research paper to complete the scoring functions
    super().__init__()

    self.att_units = att_units

    self.w1 =  tf.keras.layers.Dense( self.att_units , name = 'w1')
    self.w2 =  tf.keras.layers.Dense( self.att_units,name = 'w2')
    self.v =  tf.keras.layers.Dense(1,name = 'v')

  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    self.decoder_hidden_state = decoder_hidden_state
    self.encoder_output = encoder_output


    self.decoder_hidden_state = tf.expand_dims(self.decoder_hidden_state,axis = 1)
    score = self.v(tf.nn.tanh(
              self.w1(self.decoder_hidden_state) + self.w2(self.encoder_output)))
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * self.encoder_output
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector,attention_weights



class OneStepDecoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units  ,att_units):
      
      # Initialize decoder embedding layer, LSTM and any other objects needed
      super().__init__()
      self.tar_vocab_size = tar_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.att_units = att_units
      self.dec_emb = Embedding(tar_vocab_size,embedding_dim,trainable = True , name = 'dec_embb')
      self.dec_lstm = GRU(self.dec_units, return_state=True, return_sequences=True, name="Decoder_LSTM")
      self.dense   = Dense(self.tar_vocab_size, name = 'one_dec')
      self.attention=Attention( self.att_units)
      self.d1 = Dropout(0.3,name = 'd1')
      self.d2 = Dropout(0.3,name = 'd2')
      self.d3 = Dropout(0.3,name = 'd3')

  @tf.function
  def call(self,input_to_decoder, encoder_output, state_h):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    self.input_to_decoder = input_to_decoder
    self.encoder_output = encoder_output
    self.state_h = state_h

    #A
    target_embedd           = self.dec_emb (self.input_to_decoder)     #(batch_size,1,embedingdim)
    #B
    target_embedd = self.d1(target_embedd)

    context_vector,attention_weights=self.attention(self.state_h,self.encoder_output) #context vector shape = (batch_size,att_units)
    #C
    concated = tf.concat([  tf.expand_dims(context_vector, 1),target_embedd], -1)
    concated = self.d2(concated)

    #D
    lstm_output, hs      = self.dec_lstm(concated, initial_state=self.state_h)

    lstm_output = tf.reshape(lstm_output, (-1, lstm_output.shape[2]))
    lstm_output = self.d3(lstm_output)
    #E
    op = self.dense(lstm_output)
    #op = tf.squeeze(op,[1])
    return op,hs,attention_weights,context_vector


class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super().__init__()
      self.out_vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.att_units = att_units
      
      self.onestep = OneStepDecoder(self.out_vocab_size,self.embedding_dim ,self.input_length,self.dec_units,self.att_units)

    @tf.function
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state):


        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook

        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array

        all_outputs = tf.TensorArray(tf.float32,size =input_to_decoder.shape[1],name = 'output_arrays' )
        self.input_to_decoder = input_to_decoder
        self.encoder_output = encoder_output
        self.decoder_hidden_state = decoder_hidden_state

        for timestep in tf.range(input_to_decoder.shape[1]):
          op,hs,attention_weights,context_vector = self.onestep(self.input_to_decoder[:,timestep:timestep+1], self.encoder_output, self.decoder_hidden_state)
          self.decoder_hidden_state = hs
          all_outputs = all_outputs.write(timestep,op)
        all_outputs = tf.transpose(all_outputs.stack(),[1,0,2])
        return all_outputs
class encoder_decoder(tf.keras.Model):
  #def __init__(self,#params):
    #Intialize objects from encoder decoder
  def __init__(self,out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units,batch_size,units):

        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        super().__init__()

        self.units = units
        self.out_vocab_size = out_vocab_size
        self.embedding_size_d = embedding_size_d
        self.lstm_size_d = lstm_size_d
        self.input_length_d = input_length_d
        self.batch_size = batch_size
        self.att_units = att_units

        self.encoder = Encoder(self.units)
        print("OUT_VOCAB_SIZE = ",out_vocab_size, embedding_size_d,input_length_d,lstm_size_d,att_units)

        self.decoder = Decoder(out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units )
        #self.dense   = TimeDistributed(Dense(self.out_vocab_size, activation='softmax'))
        self.dense   = Dense(self.out_vocab_size,name = 'enc_dec_dense')



  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    self.inputs,self.outputs = data[0], data[1]
    print("="*20, "ENCODER", "="*20)
    self.encoder_h= self.encoder.initialize_states(self.batch_size)
    self.encoder_output = self.encoder(self.inputs)
    print("-"*27)
    print("ENCODER ==> OUTPUT SHAPE",self.encoder_output.shape)
    print("ENCODER ==> HIDDEN STATE SHAPE",self.encoder_h.shape)
    print("="*20, "DECODER", "="*20)
    output= self.decoder(self.outputs,self.encoder_output,self.encoder_h)
    print("-"*27)
    print("FINAL OUTPUT SHAPE",output.shape)
    print("="*50)
    print("output = ", self.dense(output))

    return output




optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  

  return tf.reduce_mean(loss_)
  #out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units,batch_size)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    verbose = 1,
    mode='min',
    save_best_only=True)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8,mode = 'min',verbose = 1,
                              patience=2, min_lr=0.0001)

model_1 = encoder_decoder(vocab_size_imp,300,76,256,64,50,256)
optimizer = tf.keras.optimizers.Adam()
model_1.compile(optimizer=optimizer,loss=loss_function, metrics = "accuracy")

train_inp = np.vstack(X_train[:,4]).astype(np.float64)  # Use np.float64 for double-precision floats
test_inp = np.vstack(X_test[:,4]).astype(np.float32)  # Use np.float32 for single-precision floats (if memory efficiency is a concern)

model_1.fit([train_inp,dec_inp ],dec_op ,validation_data= ([test_inp, dec_inp_cv],dec_op_cv),batch_size= 50,epochs  = 1,callbacks=[reduce_lr,model_checkpoint_callback] , shuffle=True)


now show with and example from the above data how and when classes are initilized and what they return what they do and show the output , when they do 

{'OOV': 1,
 'the': 2,
 'no': 3,
 'is': 4,
 'are': 5,
 '<start>': 6,
 '<end>': 7,
 'and': 8,
 'pleural': 9,
 'of': 10,
 'or': 11,
 'there': 12,
 'normal': 13,
 'heart': 14,
 'lungs': 15,
 'focal': 16,
 'size': 17,
 'within': 18,
 'in': 19,
 'pneumothora': 20,
 'normal.': 21,
 'pulmonary': 22,
 'effusion': 23,
 'pneumothora.': 24,
 'effusion.': 25,
 'limits.': 26,
 'right': 27,
 'silhouette': 28,
 'mediastinal': 29,
 'clear.': 30,
 'airspace': 31,
 'cardiomediastinal': 32,
 'left': 33,
 'clear': 34,
 'acute': 35,
 'consolidation': 36,
 'lung': 37,
 'with': 38,
 'structures': 39,
 'thoracic': 40,
 'mediastinum': 41,
 'changes': 42,
 'unremarkable.': 43,
 'a': 44,
 'bony': 45,
 'stable': 46,
 'degenerative': 47,
 'mild': 48,
 'spine.': 49,
 'without': 50,
 'large': 51,
 'appear': 52,
 'contours': 53,
 'consolidation.': 54,
 'osseous': 55,
 'cardiac': 56,
 'limits': 57,
 'size.': 58,
 '.': 59,
 'calcified': 60,
 'evidence': 61,
 'for': 62,
 'disease.': 63,
 'intact.': 64,
 'vascularity': 65